In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# NLP libraries to clean the text data
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

# Vectorization technique TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# For Splitting the dataset
from sklearn.model_selection import train_test_split

# Model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#Accuracy measuring library
from sklearn.metrics import accuracy_score

In [ ]:
file_path = '../input/fake-news-detection/data.csv'
data = pd.read_csv(file_path)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.isnull().sum()

**DATA PREPROCESSING**

INCLUDES:
* Removing the Null values
* Adding a new field
* Drop features that are not needed
* Text Processing

In [ ]:
df = data.copy()

 **Removing the Null Values**

In [ ]:
df['Body'] = df['Body'].fillna('')

In [ ]:
df.isnull().sum()

**Adding a new column**

In [ ]:
df['News'] = df['Headline']+df['Body']

In [ ]:
df.head()

In [ ]:
df.columns

**Drop features that are not needed**

In [ ]:
features_dropped = ['URLs','Headline','Body']
df = df.drop(features_dropped, axis =1)

In [ ]:
df.columns

**Text Processing**

* Remove symbols(',','-',...etc)
* Remove stop words
* Stemming

In [ ]:
ps = PorterStemmer()
def wordopt(text):
    text = re.sub('[^a-zA-Z]', ' ',text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text

In [ ]:
df['News'] = df['News'].apply(wordopt)

In [ ]:
df.head()

In [ ]:
X = df['News']
Y = df['Label']

#Split the data into training and test set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

**Vectorization**

This is used to handle our text data, by converting it into vectors

In [ ]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

**Model Fitting**

I will be fitting my data onto 3 classifications models

RandomForestClassifier
SVM
Logistic Regression
The best one amongst the 3 will be used further

In [ ]:
RFC_model = RandomForestClassifier(random_state=0)

#Fitting training set to the model
RFC_model.fit(xv_train, y_train)

#Predicting the test set results based on the model
rfc_y_pred = RFC_model.predict(xv_test)

#Calculate the accuracy score of this model
score = accuracy_score(y_test,rfc_y_pred)
print('Accuracy of RFC model is ', score)

In [ ]:

svm_model = SVC(kernel='linear')

#Fitting training set to the model
svm_model.fit(xv_train,y_train)

#Predicting the test set results based on the model
svm_y_pred = svm_model.predict(xv_test)

#Calculate the accuracy score of this model
score = accuracy_score(y_test,svm_y_pred)
print('Accuracy of SVM model is ', score)

In [ ]:
LR_model = LogisticRegression()

#Fitting training set to the model
LR_model.fit(xv_train,y_train)

#Predicting the test set results based on the model
lr_y_pred = LR_model.predict(xv_test)

#Calculate the accurracy of this model
score = accuracy_score(y_test,lr_y_pred)
print('Accuracy of LR model is ', score)

**Manual Model Testing**

In [ ]:
# As SVM is able to provide best results - SVM will be used to check the news liabilit
def fake_news_det(news):
    input_data = {"text":[news]}
    new_def_test = pd.DataFrame(input_data)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    #print(new_x_test)
    vectorized_input_data = vectorization.transform(new_x_test)
    prediction = svm_model.predict(vectorized_input_data)
    
    if prediction == 1:
        print("Not a Fake News")
    else:
        print("Fake News")

In [ ]:
fake_news_det("JetNation FanDuel League; Week 4 of readers think this story is Fact. Add your two cents.(Before Its News)Our FanDuel league is back again this week. Here are the details:$900 in total prize money. $250 to the winner. $10 to enter.Remember this is a one week league, pick your lineup against the salary cap and next week if you want to play again you can pick a completely different lineup if you want.Click this link to enter — http://fanduel.com/JetNation You can discuss this with other NY Jets fans on the Jet Nation message board. Or visit of on Facebook.Source: http://www.jetnation.com/2017/09/27/jetnation-fanduel-league-week-4/")

In [ ]:
fake_news_det("""The second Covid-19 wave in India is now on the "downswing," the Centre said on Thursday, highlighting that the current number of active cases is still "very high" and advised states and Union territories (UTs) to not let down their guards.""")

In [ ]:
fake_news_det('Teacher Sparks Outrage By Asking Kids To Make Nazi Mascots')

In [ ]:
fake_news_det("""North Korea's Other Threat: Why EMP Becomes An Extinction Level Event""")